In [1]:
import math
import numpy as np

# T_k^0, R_k(last_t), A_k, r_k(last_t)
clients = []
ready_clients = []
T = 0.
A =  (500 * 5.5 / 4.5) + math.sqrt((5.5 / 3.5) * (500 / 4.5) ** 2)
l = 1 / (100 - math.sqrt(100))
r = 3.5 * (l * A)

In [2]:
A * l, r

(8.337736223525948, 29.182076782340815)

In [3]:
r

29.182076782340815

In [4]:
import threading
import time
from IPython.display import clear_output
from threading import Thread

A_h_sum = 0
for i in range(100):
    A = (np.random.pareto(5.5) + 1) * 500
    rk = np.random.uniform(r * 0.7, r * 1.3)
#     A = 370
#     rk = r
    A_h_sum += A
    client = [T, 0., A, rk]
    clients.append(client)

# KB / msec

In [5]:
class GenerationThread(Thread):
    def __init__(self, poisson_param, power_param, val_power):
        self.poisson_param = poisson_param
        self.power_param = power_param
        self.val_power = val_power
        Thread.__init__(self)
    
    def run(self):
        global clients, e, T
        for i in range(50):
            pause = np.random.poisson(self.poisson_param)
            e.wait(timeout=pause / 1000.)
            A = (np.random.pareto(self.power_param) + 1) * self.val_power
            rk = np.random.uniform(r * 0.7, r * 1.3)
            client = [T, 0., A, rk]
            clients.append(client)
    
e = threading.Event()
        
generation = GenerationThread(100, 5.5, 500)
generation.setDaemon(True)
generation.start()

class ListeningThread(Thread):
    def __init__(self, timeout):
        self.timeout = timeout
        Thread.__init__(self)
     
    def run(self):
        global clients, e1, T, rk_mean
        last_ = 0
        while True:
            T += 1
            e.wait(timeout=self.timeout)
            try:
                k = []
                for i in range(len(clients)):
                    Tk0 = clients[i][0]
                    Rk = clients[i][1] + 1e-20
                    clients[i][3] = np.random.uniform(r * 0.7, r*1.3)
                    rk = clients[i][3]
                    k.append(rk / (T - Tk0))
#                 i = np.random.choice(np.arange(0, len(clients)), p=[ki / sum(k) for ki in k])
                i = k.index(max(k))
                clients[i][1] += min(clients[i][3], clients[i][2] - clients[i][1])
                clear_output(wait=True)
#                 print(A_, r_)
#                 print(T, len(k))
                print(len(clients))
                if clients[i][1] == clients[i][2]:
                    ready_clients.append(clients[i] + [T])
                    del clients[i]
            except:
                pass
    
e1 = threading.Event()
        
listening = ListeningThread(0.00067)
listening.setDaemon(True)
listening.start()

generation.join()
listening.stop()

1


AttributeError: 'ListeningThread' object has no attribute 'stop'

In [6]:
len(ready_clients)

1100

In [7]:
HuaweiQ = 0.
for k in ready_clients:
    HuaweiQ += k[1] / (k[-1] - k[0])
HuaweiQ /= len(ready_clients)

In [8]:
HuaweiQ

25.405201120881557

$$Q_{Huawei}(Index_{Huawei}) = 1.44 $$
$$Q_{Huawei}(Index_{Google}) = 2.08 $$
$$Q_{Huawei}(Index_{Intel, IBM}) = 2.58 $$
$$Q_{Huawei}(Index_{Old}) = 1.27 $$
$$Q_{Huawei}(Index_{Timofey}) = 9.58 $$

In [10]:
# import pickle 

# with open("HuaweiQ(HuaweiIndex).q", "wb") as f:
#     pickle.dump(HuaweiQ, f)
    
# with open("HuaweiQ(HuaweiIndex).q", "rb") as f:
#     HuaweiQ = pickle.load(f)